### ASTR 598 - Astrostatistics - Class Project - Group 1

- Tyler Gordon
- Meredith Durbin 
- Brianna Thomas
- Joachim Moeyens
- Dino Bektesevic

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from crossmatch import xmatch

from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord

% matplotlib inline

### Data Acquisition

[Google Drive for NSC files and xmatch arrays](https://drive.google.com/drive/folders/1r0LgsJ4LjUgcxUIYwPUZl-YcwxqO0z5c?usp=sharing)

[Website for HLC files](http://das.sdss.org/va/stripe_82_variability/SDSS_82_public/)

In [2]:
# Assumes there is a directory called data 
# in the same level as this notebook
DATA_DIR = "data/"
HLC_FILES = sorted(glob.glob(os.path.join(DATA_DIR, "HLC*.fits")))
NSC_FILES = sorted(glob.glob(os.path.join(DATA_DIR, "stripe82*.txt")))
# Sorting insures cross OS compatibility for list ordering

In [3]:
sorted(os.listdir(DATA_DIR))

['HLC.RA_00_to_01.fits',
 'HLC.RA_01_to_02.fits',
 'HLC.RA_02_to_03.fits',
 'HLC.RA_03_to_04.fits',
 'HLC.RA_20_to_21.fits',
 'HLC.RA_21_to_22.fits',
 'HLC.RA_22_to_23.fits',
 'HLC.RA_23_to_24.fits',
 'manxmatch_hlc_nsc_d2d.txt',
 'manxmatch_hlc_nsc_idx.txt',
 'manxmatch_nsc_hlc_d2d.txt',
 'manxmatch_nsc_hlc_idx.txt',
 'stripe82_315_ra_45_-1_3_dec_0.txt',
 'stripe82_315_ra_45_0_dec_1_3.txt',
 'xmatch_hlc_nsc_1arcsec_dist.txt',
 'xmatch_hlc_nsc_1arcsec_idx.txt',
 'xmatch_nsc_hlc_1arcsec_dist.txt',
 'xmatch_nsc_hlc_1arcsec_idx.txt']

In [4]:
HLC_FILES

['data/HLC.RA_00_to_01.fits',
 'data/HLC.RA_01_to_02.fits',
 'data/HLC.RA_02_to_03.fits',
 'data/HLC.RA_03_to_04.fits',
 'data/HLC.RA_20_to_21.fits',
 'data/HLC.RA_21_to_22.fits',
 'data/HLC.RA_22_to_23.fits',
 'data/HLC.RA_23_to_24.fits']

In [5]:
NSC_FILES

['data/stripe82_315_ra_45_-1_3_dec_0.txt',
 'data/stripe82_315_ra_45_0_dec_1_3.txt']

In [6]:
# Simple function to read in HLC and NSC files into a single dataframe

def buildHLCDataFrame(datafiles,
                      filters=["u", "g", "r", "i", "z"],
                      columns=["RA_MEAN", "DEC_MEAN",
                               "MEAN_PSFMAG", "MEAN_PSFMAG_ERR",
                               "MEAN_OBJECT_TYPE", "RA_PM_CLIP", 
                               "RA_PM_ERR_CLIP", "DEC_PM_CLIP",
                               "DEC_PM_ERR_CLIP"],
                      makeColors=True,
                      magColumnName="MEAN_PSFMAG",
                      colors=["g-i"]):
    
    dfs = []
    for datafile in datafiles:
        hdul = fits.open(datafile)
        data = hdul[1].data
        df = pd.DataFrame()

        for column in columns:
            # Assume multi-dimensional fits columns are keyed on filters
            if data[column][0].shape == (len(filters),):
                for i, filt in enumerate(filters):
                    df["{}_{}".format(column, filt)] = data[column][:, i]
            # If not multi-dimensional just add to dataframe as normal
            elif data[column][0].shape == ():
                df[column] = data[column]
            # If it isn't singular in dimension, or fits the expected number of filters
            # raise a hopefully useful error
            else:
                raise ValueError("Shape of multi-dimensional column data does not match number of filters!")

        if makeColors is True:
            for color in colors:
                # color = filt2 - filt1
                filt1 = color.split("-")[1]
                filt2 = color.split("-")[0]
                df[color] = df["{}_{}".format(magColumnName, filt2)] - df["{}_{}".format(magColumnName, filt1)]
        dfs.append(df)
    final = pd.concat(dfs)
    final.reset_index(inplace=True, drop=True)
    return final

def buildNSCDataFrame(datafiles):

    dfs = []
    for datafile in datafiles:
        df = pd.read_csv(NSC_FILES[0], sep=" ")
        dfs.append(df)
    
    final = pd.concat(dfs)
    final.reset_index(inplace=True, drop=True)
    return final

In [7]:
hlc = buildHLCDataFrame(HLC_FILES)

In [8]:
hlc.head()

,RA_MEAN,DEC_MEAN,MEAN_PSFMAG_u,MEAN_PSFMAG_g,MEAN_PSFMAG_r,MEAN_PSFMAG_i,MEAN_PSFMAG_z,MEAN_PSFMAG_ERR_u,MEAN_PSFMAG_ERR_g,MEAN_PSFMAG_ERR_r,MEAN_PSFMAG_ERR_i,MEAN_PSFMAG_ERR_z,MEAN_OBJECT_TYPE,RA_PM_CLIP,RA_PM_ERR_CLIP,DEC_PM_CLIP,DEC_PM_ERR_CLIP,g-i
0,0.000012,-0.681220,0.0000,22.679943,21.688560,21.280607,20.960501,0.000000,0.070077,0.044086,0.038800,0.177185,3.2,0.014193,0.010227,0.037836,0.010227,1.399336
1,0.000035,0.475404,0.0000,22.494812,21.485416,21.199217,20.647459,0.000000,0.041831,0.025648,0.031936,0.111785,3.0,-0.016163,0.008770,0.030977,0.008770,1.295595
2,0.000151,1.059526,0.0000,21.816391,22.191387,20.808842,20.021967,0.000000,0.097663,0.028077,0.012231,0.020769,5.4,-0.012494,0.003727,-0.006675,0.003727,1.007549
3,0.000205,-0.838721,19.7705,18.835068,18.476278,18.329546,18.270407,0.041607,0.003259,0.002162,0.002186,0.003870,6.0,0.015038,0.002609,-0.001602,0.002609,0.505522
4,0.000228,0.077162,0.0000,20.968287,19.645609,18.767252,18.273226,0.000000,0.008626,0.004235,0.003535,0.006249,6.0,-0.012199,0.003451,-0.009051,0.003451,2.201035


In [9]:
nsc = buildNSCDataFrame(NSC_FILES)

In [10]:
nsc.head()

,ra,dec,pmra,pmraerr,pmdec,pmdecerr,mjd,deltamjd,gmag,grms,...,rmag,rrms,rerr,imag,irms,ierr,class_star,fwhm,ebv,nphot
0,315.012148,-1.298594,1275.324779,1314.590162,-91.929458,1398.319357,57598.697285,10.879461,99.989998,999999.0,...,20.612076,999999.0,0.027147,99.989998,999999.0,9.99,0.978889,1.380337,0.087262,2
1,315.010754,-1.297880,-714.960501,3414.340885,-272.622409,3447.444185,57598.697285,10.879461,99.989998,999999.0,...,21.986279,999999.0,0.079821,99.989998,999999.0,9.99,0.899216,1.522221,0.087286,2
2,315.013867,-1.296287,-2339.889414,6293.682515,-2186.863370,6311.702668,57598.697285,10.879461,99.989998,999999.0,...,22.269653,999999.0,0.095987,99.989998,999999.0,9.99,0.340959,2.487719,0.087237,2
3,315.003075,-1.293646,106.992177,118.045727,18.378282,119.195841,57717.185732,360.905060,22.649281,999999.0,...,21.409351,999999.0,0.052671,99.989998,999999.0,9.99,0.866193,1.791050,0.087430,3
4,315.003150,-1.291179,-1.564847,24.141297,-6.216486,29.390321,57717.185732,360.905060,18.699413,999999.0,...,18.291037,999999.0,0.006475,99.989998,999999.0,9.99,0.984123,1.374196,0.087436,3


### Crossmatching (using DataLab crossmatch.py)

In [11]:
### 1 arcsecond in degrees
arcsecond = 0.000277778
DO_CROSSMATCH = False

In [ ]:
if DO_CROSSMATCH:
    matched_1arcsec = xmatch(hlc["RA_MEAN"].values,
                             hlc["DEC_MEAN"].values,
                             nsc["ra"].values,
                             nsc["dec"].values,
                             maxdist=arcsecond)
    reverse_matched_1arcsec = xmatch(nsc["ra"].values,
                                     nsc["dec"].values,
                                     hlc["RA_MEAN"].values,
                                     hlc["DEC_MEAN"].values,
                                     maxdist=arcsecond)
    
    np.savetxt("xmatch_hlc_nsc_1arcsec_idx.txt",  matched_1arcsec[0], fmt="%i")
    np.savetxt("xmatch_hlc_nsc_1arcsec_dist.txt", matched_1arcsec[1])
    
    np.savetxt("xmatch_nsc_hlc_1arcsec_idx.txt",  reverse_matched_1arcsec[0], fmt="%i")
    np.savetxt("xmatch_nsc_hlc_1arcsec_dist.txt", reverse_matched_1arcsec[1])
else:
    nsc2hlc_mask_1arcsec = np.loadtxt(os.path.join(DATA_DIR,"xmatch_hlc_nsc_1arcsec_idx.txt"), unpack=True)
    nsc2hlc_mask_dist_1arcsec = np.loadtxt(os.path.join(DATA_DIR,"xmatch_hlc_nsc_1arcsec_dist.txt"), unpack=True)
    
    hlc2nsc_mask_1arcsec = np.loadtxt(os.path.join(DATA_DIR,"xmatch_nsc_hlc_1arcsec_idx.txt"), unpack=True)
    hlc2nsc_mask_dist_1arcsec = np.loadtxt(os.path.join(DATA_DIR,"xmatch_nsc_hlc_1arcsec_dist.txt"), unpack=True)
    
    print("1 arcsecond max distance crossmatch found {} matched sources".format(len(nsc2hlc_mask_1arcsec)))
    print("1 arcsecond max distance reverse crossmatch found {} matched sources".format(len(hlc2nsc_mask_1arcsec)))

1 arcsecond max distance crossmatch found 1481111 matched sources
1 arcsecond max distance reverse crossmatch found 3218120 matched sources


In [ ]:
if DO_CROSSMATCH:
    matched_2arcsec = xmatch(hlc["RA_MEAN"].values,
                             hlc["DEC_MEAN"].values,
                             nsc["ra"].values,
                             nsc["dec"].values,
                             maxdist=2*arcsecond)
    reverse_matched_2arcsec = xmatch(nsc["ra"].values,
                                     nsc["dec"].values,
                                     hlc["RA_MEAN"].values,
                                     hlc["DEC_MEAN"].values,
                                     maxdist=2*arcsecond)
    
    np.savetxt("xmatch_hlc_nsc_1arcsec_idx.txt",  matched_2arcsec[0], fmt="%i")
    np.savetxt("xmatch_hlc_nsc_1arcsec_dist.txt", matched_2arcsec[1])
    
    np.savetxt("xmatch_nsc_hlc_1arcsec_idx.txt",  reverse_matched_2arcsec[0], fmt="%i")
    np.savetxt("xmatch_nsc_hlc_1arcsec_dist.txt", reverse_matched_2arcsec[1])
else:
    nsc2hlc_mask_1arcsec = np.loadtxt(os.path.join(DATA_DIR,"xmatch_hlc_nsc_2arcsec_idx.txt"), unpack=True)
    nsc2hlc_mask_dist_1arcsec = np.loadtxt(os.path.join(DATA_DIR,"xmatch_hlc_nsc_2arcsec_dist.txt"), unpack=True)
    
    hlc2nsc_mask_1arcsec = np.loadtxt(os.path.join(DATA_DIR,"xmatch_nsc_hlc_2arcsec_idx.txt"), unpack=True)
    hlc2nsc_mask_dist_1arcsec = np.loadtxt(os.path.join(DATA_DIR,"xmatch_nsc_hlc_2arcsec_dist.txt"), unpack=True)
    
    print("1 arcsecond max distance crossmatch found {} matched sources".format(len(nsc2hlc_mask_2arcsec)))
    print("1 arcsecond max distance reverse crossmatch found {} matched sources".format(len(hlc2nsc_mask_2arcsec)))

### Crossmatching (using AstroPy match to catalog)

In [ ]:
man_nsc2hlc_mask = np.loadtxt(os.path.join(DATA_DIR,"manxmatch_hlc_nsc_idx.txt"), unpack=True)
man_nsc2hlc_mask_dist = np.loadtxt(os.path.join(DATA_DIR,"manxmatch_nsc_hlc_d2d.txt"), unpack=True)
    
man_hlc2nsc_mask = np.loadtxt(os.path.join(DATA_DIR,"manxmatch_hlc_nsc_idx.txt"), unpack=True)
man_hlc2nsc_mask_dist = np.loadtxt(os.path.join(DATA_DIR,"manxmatch_hlc_nsc_d2d.txt"), unpack=True)
    
print("1 arcsecond max distance crossmatch found {} matched sources".format(len(man_nsc2hlc_mask)))
print("1 arcsecond max distance reverse crossmatch found {} matched sources".format(len(man_hlc2nsc_mask)))

In [ ]:
hlcmask = man_hlc2nsc_mask_dist < arcsecond
maskedhlc = hlc.iloc[hlcmask]
maskednsc = nsc.iloc[man_hlc2nsc_mask[hlcmask]]
print len(hlcmask)
print len(maskedhlc), len(maskednsc)

In [ ]:
maskedhlc

In [ ]:
maskednsc

### Data Cleaning and Proper Motion Selection

### Chi-Squared Calculation

In [ ]:
hlcrapm = maskedhlc["RA_PM_CLIP"].values
hlcrapmerr = maskedhlc["RA_PM_ERR_CLIP"].values
nscrapm = maskednsc["pmra"].values
nscrapmerr = maskednsc["pmraerr"].values
diffra = hlcrapm-(nscrapm/1000.)
sigmai = np.sqrt(  hlcrapmerr**2 + (nscrapmerr/1000.)**2 )
norm = 1/float(len(hlcrapm)-1)
chi2i = np.sqrt(norm*( diffra/sigmai )**2)
plt.plot(chi2i)
plt.show()

### Plotting

- Chi-squared distribution as a function of Ra and Dec
- Proper motion as a function of RA and Dec
- Others...

In [ ]:
import astroML as aml
from astroML.plotting import setup_text_plots
from astroML.stats import binned_statistic_2d

setup_text_plots(fontsize=16, usetex=True)

num_bins=100

N, xedge, yedge = binned_statistic_2d(maskednsc["ra"], maskednsc["dec"], maskednsc["dec"], 'count', bins=num_bins)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 8))
ax[0].imshow(N, origin='lower', aspect="auto", extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]])
ax[0].set_ylim(-0.18, 0.02)

i = ax[1].imshow(N, origin='lower', aspect="auto", extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]])
ax[1].set_ylim(-1.28, -1.09)

ax[0].set_ylabel("Dec")
ax[1].set_ylabel("Dec")
ax[1].set_xlabel("Ra")
plt.colorbar(mappable=i, ax=ax[1], pad=0.14, orientation='horizontal', label=r"$\chi_i$")
plt.show()

In [ ]:
mnscra = maskednsc["ra"].values
mnscdec = maskednsc["dec"].values

print "Number of objects with -1<dec<-0.15: {0}".format(len(mnscdec[ (mnscdec<-0.15) & (mnscdec>-1) ]))

### Attempt at finding matching indices from a constrained crossmatch 

Remakably useless right now. Interesting lack of ~6000 objects is apparent after a `match_to_catalog_sky` with constrained `maxdist` that does not appear when manually fitlering for the constrain.

In [ ]:
d = np.unique(hlc2nsc_mask_1arcsec)
e = np.unique(nsc2hlc_mask_1arcsec)
print "Number of unique matches for HLC and NSC respectively: {0}, {1}".format(len(hlc.iloc[d]), len(nsc.iloc[nsc2hlc_mask_1arcsec]))
print "Discrepancy between xmatch and reverse xmatch: {0}".format(len(hlc.iloc[d]) - len(nsc.iloc[nsc2hlc_mask_1arcsec]))

In [ ]:
hlc.iloc[d]

In [ ]:
nsc.iloc[nsc2hlc_mask_1arcsec]

In [ ]:
a = nsc.iloc[nsc2hlc_mask_1arcsec]
b = a["dec"].values
len(b[(b<-0.15) & (b>-1)])

In [ ]:
import astroML as aml
from astroML.plotting import setup_text_plots
from astroML.stats import binned_statistic_2d

setup_text_plots(fontsize=16, usetex=True)

num_bins=100
maskedhlc = hlc.iloc[d]
maskednsc = nsc.iloc[nsc2hlc_mask_1arcsec]
nscra = nsc.iloc[nsc2hlc_mask_1arcsec]["ra"].values
nscdec = nsc.iloc[nsc2hlc_mask_1arcsec]["dec"].values

N, xedge, yedge = binned_statistic_2d(maskednsc["ra"], maskednsc["dec"], maskednsc["dec"], 'count', bins=num_bins)

In [ ]:
plt.scatter(nscdec, nscra)
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 8))
ax[0].imshow(N, origin='lower', aspect="auto", extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]])
ax[0].set_ylim(-0.18, 0.02)

i = ax[1].imshow(N, origin='lower', aspect="auto", extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]])
ax[1].set_ylim(-1.28, -1.09)

ax[0].set_ylabel("Dec")
ax[1].set_ylabel("Dec")
ax[1].set_xlabel("Ra")
plt.colorbar(mappable=i, ax=ax[1], pad=0.14, orientation='horizontal', label="N")
plt.show()